In this notebook i have done some data expolration & feature analysis using Chi-Square Test for Feature Selection &  SVM-based feature selection algorithm.

Then i used different models to predict the wine quality 

1.Random Forest
2.SVC

**If you find this notebook useful then please upvote.**

In [ ]:
#import all the necessary packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
winedb=pd.read_csv(r"/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")

In [ ]:
winedb.head()

In [ ]:
winedb.describe()

In [ ]:
#Plot the corr plot to see the corelation between variables

d=pd.DataFrame(data=winedb)
corr=d.corr()
f, ax = plt.subplots(figsize=(10, 6))
hm = sns.heatmap(round(corr,2), annot=True, ax=ax, cmap="gray",fmt='.2f',
            linewidths=.05)
f.subplots_adjust(top=0.93)
t= f.suptitle('Correlation HeatMap', fontsize=14)

Alcohoal has the highest positive corelation(0.48) with the quality of the wine 

# **Preprocessing of Data Before Modelling**

Make the binary classification on the responce variable as per the tips return in data context

Divide wine into good & Bad by giving the limit for the quality

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [ ]:
#The bins parameter tells you the number of bins that your data will be divided into.
bins = (2, 6.5, 8)
group_names = ['bad', 'good']
winedb['quality'] = pd.cut(winedb['quality'], bins = bins, labels = group_names)

In [ ]:
#Assign labels
label_quality = LabelEncoder()
#Bad becomes 0 and good becomes 1 
winedb['quality'] = label_quality.fit_transform(winedb['quality'])

In [ ]:
winedb['quality'].value_counts()

In [ ]:
#Plot the counts 
sns.countplot(winedb['quality'])

In [ ]:
#Seperate the dataset
Y=winedb['quality']
X = winedb.iloc[:,:11]

In [ ]:
#Create test train split data
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X, Y, test_size = 0.2, random_state = 42)

Feature selection using the chi-square test

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [ ]:
test = SelectKBest(score_func  = chi2, k ='all')

In [ ]:
fitted = test.fit(X,Y)

In [ ]:
list(fitted.scores_)

In [ ]:
colnames =  list(X.columns)
X  = np.array(X)

In [ ]:
df_1 = pd.DataFrame({'Feature_name':list(colnames),'Feature_score':list(fitted.scores_)}) 
df_1
df_1.sort_values(['Feature_score'],ascending = False)

Feature selection using the RFE on SVM

In [ ]:
from sklearn.feature_selection import RFE  ##runs on top of some algorithm ..we can run on top of randomforest and DT also)

In [ ]:
from sklearn.svm import LinearSVC #(running on SVM)

In [ ]:
svm = LinearSVC()

In [ ]:
rfe= RFE(svm, 10)  ## keep increasing the number to get the importance in order


In [ ]:
rfe.fit(X, Y)

In [ ]:
print(rfe.support_)

In [ ]:
df2 = pd.DataFrame( { "Feature Names": colnames , "Importance" : list(rfe.support_)})
df2.sort_values(['Importance'], ascending = False )

In [ ]:
#Trying the machine learning algorithms
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(X_train, Y_train)
pred_rfc = rfc.predict(X_test)

In [ ]:
#Let's see how our model performed
print(classification_report(Y_test, pred_rfc))

In [ ]:
print(confusion_matrix(Y_test, pred_rfc))

Accuracy for Random Forest is 90%

In [ ]:
#SVM
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train, Y_train)
pred_svc = svc.predict(X_test)

In [ ]:
print(classification_report(Y_test, pred_svc))

Accuracy for the SVC is 86%